In [105]:
import pandas as pd
import json
import geopandas
import fiona 
import geocoder

## Data preprocessing 
### 1. Dataset "Studierendenzahl nach Bundesland, Nationalität und Geschlecht WS 1998/99 - WS 2016/17" (DS1)

In [66]:
DS1 = pd.read_csv('Studierende_BundesländerSemesterNationalitätGeschlecht.csv',sep=';', encoding='latin-1')

#### Original dataset DS1

In [13]:
DS1

,GENESIS-Tabelle: 21311-0005,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,"Studierende: Bundesländer, Semester, Nationali...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geschlecht,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Statistik der Studenten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Studierende (Anzahl),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Deutsche,Deutsche,Deutsche,Ausländer,Ausländer,Ausländer,Insgesamt,Insgesamt,Insgesamt
5,NaN,NaN,männlich,weiblich,Insgesamt,männlich,weiblich,Insgesamt,männlich,weiblich,Insgesamt
6,Baden-Württemberg,WS 1998/99,97587,69435,167022,11624,9866,21490,109211,79301,188512
7,Baden-Württemberg,WS 1999/00,94622,70791,165413,12781,10961,23742,107403,81752,189155
8,Baden-Württemberg,WS 2000/01,95519,73934,169453,13977,12256,26233,109496,86190,195686
9,Baden-Württemberg,WS 2001/02,97923,79056,176979,15160,14062,29222,113083,93118,206201


In [15]:
# cut empty rows and reset index
DS1 = DS1.iloc[4:-3]
DS1 = DS1.reset_index(drop = True)

# rename columns and reset index
DS1.columns.values[0] = 'Bundesland'
DS1.columns.values[1] = 'Semester'
for i in range(2,11):
    DS1.columns.values[i] = DS1.iloc[0,i] + ', ' + DS1.iloc[1,i]
DS1 = DS1.reset_index(drop = True)

# cut rows containing column names 
DS1 = DS1.iloc[2:]

# convert object type into int type
DS1[DS1.columns[2:11]] = DS1[DS1.columns[2:11]].astype(int)

# remove non-numeric values
DS1.Semester.replace(regex='WS ', value='', inplace=True)
DS1 = DS1.reset_index(drop = True)

#### Dataset DS1 after preprocessing

In [16]:
DS1

,Bundesland,Semester,"Deutsche, männlich","Deutsche, weiblich","Deutsche, Insgesamt","Ausländer, männlich","Ausländer, weiblich","Ausländer, Insgesamt","Insgesamt, männlich","Insgesamt, weiblich","Insgesamt, Insgesamt"
0,Baden-Württemberg,1998/99,97587,69435,167022,11624,9866,21490,109211,79301,188512
1,Baden-Württemberg,1999/00,94622,70791,165413,12781,10961,23742,107403,81752,189155
2,Baden-Württemberg,2000/01,95519,73934,169453,13977,12256,26233,109496,86190,195686
3,Baden-Württemberg,2001/02,97923,79056,176979,15160,14062,29222,113083,93118,206201
4,Baden-Württemberg,2002/03,100344,84873,185217,16785,15782,32567,117129,100655,217784
5,Baden-Württemberg,2003/04,105687,90327,196014,17949,17359,35308,123636,107686,231322
6,Baden-Württemberg,2004/05,109208,94037,203245,18219,18330,36549,127427,112367,239794
7,Baden-Württemberg,2005/06,111530,96707,208237,18093,18288,36381,129623,114995,244618
8,Baden-Württemberg,2006/07,111893,97465,209358,17761,18024,35785,129654,115489,245143
9,Baden-Württemberg,2007/08,107295,93066,200361,16448,16643,33091,123743,109709,233452


#### Save dataset DS1 as pickle format

In [63]:
DS1.to_pickle('students_bundesland_gender_foreigner_ws1998_99_ws2016_17.pkl')

### Prepare dataset for tooltips
#### Read geojson file with Germanys states bounds

In [56]:
with open('geo_germany.geojson') as data_file:
    state_geo = json.load(data_file)

In [57]:
state_geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 0,
   'properties': {'ID_0': 86,
    'ISO': 'DEU',
    'NAME_0': 'Germany',
    'ID_1': 1,
    'NAME_1': 'Baden-Württemberg',
    'NL_NAME_1': None,
    'VARNAME_1': None,
    'TYPE_1': 'Land',
    'ENGTYPE_1': 'State',
    'style': {'fillColor': '#b30000',
     'color': 'black',
     'weight': 1,
     'opacity': 0.2,
     'fillOpacity': 0.7},
    'highlight': {'weight': 3,
     'color': 'black',
     'fillOpacity': 0.8999999999999999}},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[9.650460243225211, 49.7763404846192],
       [9.656839370727539, 49.761451721191406],
       [9.640399932861612, 49.75014114379883],
       [9.652028083801326, 49.742759704589844],
       [9.646539688110352, 49.738990783691406],
       [9.652549743652287, 49.73157119750988],
       [9.64107894897461, 49.735221862793196],
       [9.64140987396246, 49.727748870849894],
       [9.630180358886776, 49.727668762206974],
    

#### Convert geojson to GeoDataFrame format for the following join with dataset DS1

In [58]:
GDF = geopandas.GeoDataFrame.from_features(state_geo['features'])

In [59]:
GDF

,ENGTYPE_1,ID_0,ID_1,ISO,NAME_0,NAME_1,NL_NAME_1,TYPE_1,VARNAME_1,geometry,highlight,style
0,State,86,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
1,State,86,2,DEU,Germany,Bayern,None,Land,Bavaria,"POLYGON ((10.13385963439958 50.54999923706066,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
2,State,86,3,DEU,Germany,Berlin,None,Land,None,"POLYGON ((13.16180992126476 52.59442138671869,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
3,State,86,4,DEU,Germany,Brandenburg,None,Land,None,"POLYGON ((13.87950801849371 53.50106811523443,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
4,State,86,5,DEU,Germany,Bremen,None,Land,None,"POLYGON ((8.98544883728033 53.12821960449224, ...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
5,State,86,6,DEU,Germany,Hamburg,None,Land,None,"POLYGON ((10.07161712646484 53.71823120117182,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
6,State,86,7,DEU,Germany,Hessen,None,Land,Hesse,"POLYGON ((9.498769760131779 51.63151931762695,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
7,State,86,8,DEU,Germany,Mecklenburg-Vorpommern,None,Land,Mecklenburg-West Pomerania,(POLYGON ((12.51972103118896 54.48430633544945...,"{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
8,State,86,9,DEU,Germany,Niedersachsen,None,Land,Lower Saxony,"(POLYGON ((8.680833816528661 53.8920822143557,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
9,State,86,10,DEU,Germany,Nordrhein-Westfalen,None,Land,North Rhine-Westphalia,"POLYGON ((8.666278839111385 52.52527999877924,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."


#### Join DS1 and GDF

In [60]:
result = DS1.merge(GDF, left_on='Bundesland', right_on='NAME_1')
GDT = geopandas.GeoDataFrame(result)
GDT.crs = fiona.crs.from_epsg(4326)

In [61]:
GDT

,Bundesland,Semester,"Deutsche, männlich","Deutsche, weiblich","Deutsche, Insgesamt","Ausländer, männlich","Ausländer, weiblich","Ausländer, Insgesamt","Insgesamt, männlich","Insgesamt, weiblich",...,ID_1,ISO,NAME_0,NAME_1,NL_NAME_1,TYPE_1,VARNAME_1,geometry,highlight,style
0,Baden-Württemberg,1998/99,97587,69435,167022,11624,9866,21490,109211,79301,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
1,Baden-Württemberg,1999/00,94622,70791,165413,12781,10961,23742,107403,81752,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
2,Baden-Württemberg,2000/01,95519,73934,169453,13977,12256,26233,109496,86190,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
3,Baden-Württemberg,2001/02,97923,79056,176979,15160,14062,29222,113083,93118,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
4,Baden-Württemberg,2002/03,100344,84873,185217,16785,15782,32567,117129,100655,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
5,Baden-Württemberg,2003/04,105687,90327,196014,17949,17359,35308,123636,107686,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
6,Baden-Württemberg,2004/05,109208,94037,203245,18219,18330,36549,127427,112367,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
7,Baden-Württemberg,2005/06,111530,96707,208237,18093,18288,36381,129623,114995,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
8,Baden-Württemberg,2006/07,111893,97465,209358,17761,18024,35785,129654,115489,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."
9,Baden-Württemberg,2007/08,107295,93066,200361,16448,16643,33091,123743,109709,...,1,DEU,Germany,Baden-Württemberg,None,Land,None,"(POLYGON ((9.650460243225211 49.7763404846192,...","{'weight': 3, 'color': 'black', 'fillOpacity':...","{'fillColor': '#b30000', 'color': 'black', 'we..."


#### Save dataset for tooltips (GDT) as pickle format

In [62]:
GDT.to_pickle('tooltip_geojson_fiona.pkl')

### Prepare population dataset (DSB)

In [85]:
DSB = pd.read_excel('bevölkerung_bundesland_1991_2017.xlsx')

#### Original population dataset (DSB)

In [74]:
DSB

,Einwohner in Deutschland nach Bundesländern*),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,1.000 Personen (Jahresdurchschnitt),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jahr,BW,BY,BE,BB,HB,HH,HE,MV,NI,NW,RP,SL,SN,ST,SH,TH,D
2,1991,9904,11517.8,3436.2,2559.8,682.4,1658.7,5797.9,1907.2,7427.7,17421.3,3792.1,1074.5,4719.5,2847.7,2636.1,2590.6,79973.4
3,1992,10050.4,11668.8,3444.4,2540.2,684,1673,5872.2,1876.5,7515.1,17568.6,3850.3,1079,4653.6,2807.4,2660.2,2555.9,80499.8
4,1993,10149.3,11792.7,3450.7,2535.5,683.3,1685.9,5931.7,1851.2,7594.3,17676,3902.1,1081.9,4613.8,2782.8,2680.8,2534.4,80946.5
5,1994,10195.3,11859.9,3445.1,2530.5,679.9,1690.3,5955.4,1833.6,7656.2,17728.4,3936.9,1081.2,4581.5,2762.1,2692.9,2518.3,81147.5
6,1995,10223.1,11917,3434.2,2530.6,677.8,1689,5971.7,1822.2,7714.6,17779.9,3962.4,1080.3,4556.6,2740.7,2706,2501.7,81307.7
7,1996,10260.1,11970.3,3418.4,2537.1,676.2,1686.2,5990.3,1813.3,7756.9,17830.8,3986.4,1079.5,4532.8,2721,2721.1,2486.4,81466.4
8,1997,10285.1,11999.3,3386,2550.2,672.8,1680.8,5996.5,1804.2,7782.2,17856.9,4005.9,1076.9,4506.2,2700.4,2735.1,2471.5,81509.9
9,1998,10297.4,12013,3346.2,2565.7,667.4,1672.9,5995.8,1793.6,7800.6,17856.1,4017.6,1071.1,4473.7,2673.9,2745.5,2455.5,81446


In [86]:
# cut empty rows
DSB = DSB.iloc[1:-4]

# rename columns
DSB.columns = DSB.iloc[0]

# reset index
DSB = DSB.reset_index(drop = True)

# remove unnecessary data
DSB = DSB.iloc[8:27]
DSB = DSB.reset_index(drop = True)

# convert object type into int type
DSB[DSB.columns[1:18]] = DSB[DSB.columns[1:18]].astype(float)

In [88]:
# rename columns
for i, item in enumerate(state_geo.get('features')):
    DSB.columns.values[i+1] = item.get('properties').get('NAME_1')

In [91]:
# remove column with totals
DSB = DSB.iloc[:,:-1]

#### Dataset DSB after preprocessing

In [92]:
DSB

1,Jahr,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen-Anhalt,Sachsen,Schleswig-Holstein,Thüringen
0,1998,10297.4,12013.0,3346.2,2565.7,667.4,1672.9,5995.8,1793.6,7800.6,17856.1,4017.6,1071.1,4473.7,2673.9,2745.5,2455.5
1,1999,10323.5,12049.7,3316.8,2577.2,661.6,1668.9,6001.2,1783.0,7820.4,17853.8,4023.6,1065.7,4438.1,2645.5,2754.4,2439.1
2,2000,10359.2,12113.9,3298.8,2580.6,657.2,1672.5,6013.0,1770.0,7843.1,17856.1,4027.9,1062.2,4402.0,2614.0,2764.7,2421.3
3,2001,10408.2,12193.5,3290.2,2574.4,655.1,1679.3,6021.0,1753.8,7863.8,17867.4,4036.5,1058.9,4359.9,2577.9,2776.7,2400.4
4,2002,10463.3,12264.2,3286.1,2562.4,655.6,1682.0,6028.0,1736.8,7882.8,17886.0,4047.6,1056.3,4317.2,2542.7,2788.6,2378.8
5,2003,10496.2,12303.6,3277.1,2551.1,656.8,1681.8,6029.0,1721.5,7893.5,17885.3,4052.0,1053.1,4281.7,2511.8,2796.6,2357.6
6,2004,10511.6,12324.7,3265.9,2541.7,656.9,1680.6,6027.0,1707.5,7896.4,17870.6,4053.2,1048.1,4251.2,2482.6,2801.2,2337.3
7,2005,10520.7,12340.3,3260.5,2532.1,656.7,1681.1,6023.4,1693.6,7889.8,17845.6,4052.9,1041.8,4223.4,2454.1,2804.7,2316.1
8,2006,10519.0,12357.7,3259.8,2520.2,656.6,1686.7,6007.5,1679.3,7874.7,17808.5,4048.3,1034.5,4196.2,2426.0,2806.0,2292.1
9,2007,10513.7,12376.3,3261.3,2506.4,656.0,1696.2,5992.9,1664.1,7857.9,17763.8,4041.3,1026.9,4165.6,2395.7,2806.8,2267.5


#### Save population dataset (DSB) as pickle format

In [93]:
DSB.to_pickle('bevoelkerung_1998_2016.pkl')

### 2. Dataset "Hochschulen nach Gründungsjahr von 1386 bis 2017" (DS2)

In [101]:
DS2 = pd.read_excel('hs_liste.xlsx')

#### Original dataset DS2

In [97]:
DS2

,Hs-Nr.,Hochschulkurzname,Hochschulname,Hochschultyp,Trägerschaft,Bundesland,Anzahl Studierende,Gründungsjahr,Promotionsrecht,Habilitationsrecht,...,PLZ,Ort,Postfach,Postleitzahl (Postanschrift),Ort (Postanschrift),Telefonvorwahl,Telefon,Fax,Home Page,Mitglied HRK
0,2,Aachen FH,Fachhochschule Aachen,Fachhochschulen / HAW,öffentlich-rechtlich,Nordrhein-Westfalen,14083.0,1971.0,Nein,Nein,...,52066.0,Aachen,100560,52005.0,Aachen,241,6009-0,6009-51090,http://www.fh-aachen.de,1.0
1,1,Aachen TH,Rheinisch-Westfälische Technische Hochschule A...,Universitäten,öffentlich-rechtlich,Nordrhein-Westfalen,45403.0,1870.0,Ja,Ja,...,52062.0,Aachen,NaN,52056.0,Aachen,241,80-1,80-92312,http://www.rwth-aachen.de,1.0
2,3,Aalen H,Hochschule Aalen - Technik und Wirtschaft,Fachhochschulen / HAW,öffentlich-rechtlich,Baden-Württemberg,5928.0,1962.0,Nein,Nein,...,73430.0,Aalen,1728,73428.0,Aalen,7361,576-0,576-2250,http://www.hs-aalen.de,1.0
3,4,Albstadt-Sigmaringen H,Hochschule Albstadt-Sigmaringen,Fachhochschulen / HAW,öffentlich-rechtlich,Baden-Württemberg,3545.0,1971.0,Nein,Nein,...,72488.0,Sigmaringen,1254,72481.0,Sigmaringen,7571,732-0,732-8229,www.hs-albsig.de,1.0
4,349,Alfter HfK,Alanus Hochschule,Kunst- und Musikhochschulen,"privat, staatlich anerkannt",Nordrhein-Westfalen,1575.0,1973.0,Ja,Nein,...,53347.0,Alfter,NaN,NaN,NaN,2222,9321-0,9321-21,https://www.alanus.edu,0.0
5,280,Amberg-Weiden OTH,Ostbayerische Technische Hochschule Amberg-Weiden,Fachhochschulen / HAW,öffentlich-rechtlich,Bayern,3096.0,1994.0,Nein,Nein,...,92224.0,Amberg,1462,92204.0,Amberg,9621,482-0,482-4991,http://www.oth-aw.de/,1.0
6,5,Anhalt H,Hochschule Anhalt - Anhalt University of Appli...,Fachhochschulen / HAW,öffentlich-rechtlich,Sachsen-Anhalt,7159.0,1991.0,Nein,Nein,...,6366.0,Köthen,1458,6354.0,Köthen,3496,67-1000,67-1099,http://www.hs-anhalt.de,1.0
7,312,Ansbach H,Hochschule für angewandte Wissenschaften Ansbach,Fachhochschulen / HAW,öffentlich-rechtlich,Bayern,2902.0,1996.0,Nein,Nein,...,91522.0,Ansbach,1963,91510.0,Ansbach,981,4877-0,4877-188,http://www.hs-ansbach.de,1.0
8,333,Aschaffenburg H,Hochschule für angewandte Wissenschaften - Fac...,Fachhochschulen / HAW,öffentlich-rechtlich,Bayern,3296.0,1995.0,Nein,Nein,...,63743.0,Aschaffenburg,NaN,NaN,NaN,6021,4206-0,4206-600,http://www.h-ab.de,1.0
9,7,Augsburg H,Hochschule für angewandte Wissenschaften Augsb...,Fachhochschulen / HAW,öffentlich-rechtlich,Bayern,6232.0,1710.0,Nein,Nein,...,86161.0,Augsburg,110605,86031.0,Augsburg,821,5586-0,5586-3222,http://www.hs-augsburg.de/,1.0


#### Clean dataset for further usage

In [102]:
# Keep only relevant columns 
DS2 = DS2[['Hochschulname', 'Hochschulkurzname', 'Hochschultyp', 'Bundesland', \
                   'Gründungsjahr', 'Ort', 'PLZ', 'Strasse']]

# Cut last three unnecessary rows
DS2 = DS2.iloc[:-3]

# Convert from float to int type
DS2[['PLZ', 'Gründungsjahr']] = DS2[['PLZ', 'Gründungsjahr']].astype(int)

#### Dataset DS2 after cleaning

In [103]:
DS2

,Hochschulname,Hochschulkurzname,Hochschultyp,Bundesland,Gründungsjahr,Ort,PLZ,Strasse
0,Fachhochschule Aachen,Aachen FH,Fachhochschulen / HAW,Nordrhein-Westfalen,1971,Aachen,52066,Bayernallee 11
1,Rheinisch-Westfälische Technische Hochschule A...,Aachen TH,Universitäten,Nordrhein-Westfalen,1870,Aachen,52062,Templergraben 55
2,Hochschule Aalen - Technik und Wirtschaft,Aalen H,Fachhochschulen / HAW,Baden-Württemberg,1962,Aalen,73430,Beethovenstraﬂe 1
3,Hochschule Albstadt-Sigmaringen,Albstadt-Sigmaringen H,Fachhochschulen / HAW,Baden-Württemberg,1971,Sigmaringen,72488,Anton-Günther-Straﬂe 51
4,Alanus Hochschule,Alfter HfK,Kunst- und Musikhochschulen,Nordrhein-Westfalen,1973,Alfter,53347,Villestraﬂe 3
5,Ostbayerische Technische Hochschule Amberg-Weiden,Amberg-Weiden OTH,Fachhochschulen / HAW,Bayern,1994,Amberg,92224,Kaiser-Wilhelm-Ring 23
6,Hochschule Anhalt - Anhalt University of Appli...,Anhalt H,Fachhochschulen / HAW,Sachsen-Anhalt,1991,Köthen,6366,Bernburger Straﬂe 55
7,Hochschule für angewandte Wissenschaften Ansbach,Ansbach H,Fachhochschulen / HAW,Bayern,1996,Ansbach,91522,Residenzstraﬂe 8
8,Hochschule für angewandte Wissenschaften - Fac...,Aschaffenburg H,Fachhochschulen / HAW,Bayern,1995,Aschaffenburg,63743,Würzburger Straﬂe 45
9,Hochschule für angewandte Wissenschaften Augsb...,Augsburg H,Fachhochschulen / HAW,Bayern,1710,Augsburg,86161,An der Hochschule 1


#### Using universities addresses get they  geocoordinates via HERE API. 

In [106]:
lat = []
lon = []
for street, code, city in zip(DS2.Strasse, DS2.PLZ, DS2.Ort):
    adress = street + ', ' + str(code) + ', ' + city
    g = geocoder.here(adress,
                       app_id='<app-id>',
                       app_code='<app-code>')
    lat.append(g.lat)
    lon.append(g.lng)

In [107]:
# Add coordinates to dataset DS2
DS2['lat'] = lat
DS2['lon'] = lon

#### Save enriched dataset DS2 as pickle format

In [134]:
DS2.to_pickle("hochschule_koordinaten.pkl")

### 3. Dataset "Studierende nach Geschlecht, Land des Studienortes und Land des Erwerbs der Hochschulzugangsberechtigung WS 2006/07 - WS 2017/18" (DS3)

In [120]:
DS3 = pd.read_excel('Tabelle-2532.xls')

#### Original dataset DS3

In [115]:
DS3

,Tab.2.5.32,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Studierende nach Geschlecht, Land des Studieno...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Land des Erwerbs der Hochschulzugangsberechtigung,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Land des Erwerbs der Hochschulzugangsberechtigung,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insgesamt,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,...,Nieder-\nsachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen,Ohne Angabe,Ausland
5,Winter-semester,Land des Studienortes,Geschlecht,Anzahl,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Anzahl,NaN,NaN,NaN,NaN
6,2006/2007,Baden-Württemberg,Insgesamt,245143,158162,12929,1518,1177,470,920,...,4408,8897,10450,1478,2018,1000,1554,1573,54,29364
7,NaN,NaN,Männlich,129654,86446,7149,672,486,255,471,...,2322,4646,5420,643,836,406,850,686,15,13956
8,NaN,NaN,Weiblich,115489,71716,5780,846,691,215,449,...,2086,4251,5030,835,1182,594,704,887,39,15408
9,NaN,Bayern,Insgesamt,257898,18875,179532,1608,1368,332,800,...,4044,7426,2330,496,4583,1303,1329,3788,8,23850


#### Prepare data for further usage

In [123]:
# rename columns
DS3.rename(columns={ DS3.columns[0]: "WS", 
                    DS3.columns[1]: "Bundesland_Studienort", 
                    DS3.columns[2]: "Geschlecht" }, inplace=True)
for i in range(3,22):
    DS3.columns.values[i] = DS3.iloc[4,i]
DS3.rename(columns={"Nieder-\nsachsen": "Niedersachsen"},
          inplace=True)

# cut empty rows
DS3 = DS3.iloc[6:-14]

# fill values in first and second columns 
DS3[['WS','Bundesland_Studienort']] = DS3[['WS','Bundesland_Studienort']].ffill()

# convert columns type to int
DS3.loc[:,3:-2] = DS3.iloc[:,3:-2].astype(int)
DS3 = DS3.reset_index(drop = True)

#### Dataset DS3 after preprocessing

In [124]:
DS3

,WS,Bundesland_Studienort,Geschlecht,Insgesamt,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,...,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen,Ohne Angabe,Ausland
0,2006/2007,Baden-Württemberg,Insgesamt,245143,158162,12929,1518,1177,470,920,...,4408,8897,10450,1478,2018,1000,1554,1573,54,29364
1,2006/2007,Baden-Württemberg,Männlich,129654,86446,7149,672,486,255,471,...,2322,4646,5420,643,836,406,850,686,15,13956
2,2006/2007,Baden-Württemberg,Weiblich,115489,71716,5780,846,691,215,449,...,2086,4251,5030,835,1182,594,704,887,39,15408
3,2006/2007,Bayern,Insgesamt,257898,18875,179532,1608,1368,332,800,...,4044,7426,2330,496,4583,1303,1329,3788,8,23850
4,2006/2007,Bayern,Männlich,131650,9367,95030,750,626,159,405,...,2063,3866,1244,236,1777,593,719,1546,8,10145
5,2006/2007,Bayern,Weiblich,126248,9508,84502,858,742,173,395,...,1981,3560,1086,260,2806,710,610,2242,-,13705
6,2006/2007,Berlin,Insgesamt,132822,6055,4302,59524,14222,726,1353,...,4967,7477,1295,331,3169,2590,1927,1693,1,16800
7,2006/2007,Berlin,Männlich,66840,3228,2263,31681,6858,360,630,...,2513,3763,676,167,1271,1102,999,708,1,7589
8,2006/2007,Berlin,Weiblich,65982,2827,2039,27843,7364,366,723,...,2454,3714,619,164,1898,1488,928,985,-,9211
9,2006/2007,Brandenburg,Insgesamt,42331,1012,690,10089,16792,111,265,...,1034,1281,234,38,2169,1316,386,560,2,4866


#### Create dataset for tooltips

In [126]:
DS3_T = DS3.merge(GDF, left_on='Bundesland_Studienort', right_on='NAME_1')
DS3_T = geopandas.GeoDataFrame(DS3_T)
DS3_T.crs = fiona.crs.from_epsg(4326)

#### Save datasets DS3 and DS3_T as pickle format

In [125]:
DS3.to_pickle('students_gender_study_place_vs_study_permission_ws2006_07_ws2017_18.pkl')

In [132]:
DS3_T.to_pickle("tooltip_place_of_study_geojson_fiona.pkl")